# Ferramenta Interativa de Análise Astronômica

In [ ]:
# --- Imports ---
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
from datetime import date
import warnings

# Suprimir avisos comuns da AstroPy sobre tempo futuro
warnings.filterwarnings('ignore', category=erfa.core.ErfaWarning, message='.*ERFA function.*dubious year.*')

from src.config import *
from src.location import get_location_from_city, set_timezone_for_sao_paulo
from src.targets import get_target_skycoords, registrar_alvos_sistema_solar
from src.analysis import (
    calculate_nightly_events, 
    analyze_target_visibility_for_night, 
    check_hemisphere_visibility, 
    analyze_moon_impact,
    get_weather_forecast,
    analyze_year_visibility # Novo import
)
from src.plotting import (
    plot_target_visibility, 
    plot_sky_map,
    plot_yearly_visibility # Novo import
)

print("Módulos e dependências carregados.")

## 1. Análise Noturna Detalhada

In [ ]:
# --- Widgets da UI (como antes) ---
location_method = widgets.RadioButtons(options=['Cidade', 'Coordenadas'], description='Local:', value='Cidade', layout={'width': 'max-content'})
city_name = widgets.Text(description='Cidade:', placeholder='Ex: Uberaba, Brazil')
lat_lon_input = widgets.Text(description='Lat,Lon:', placeholder='Ex: -19.74, -47.93')
box_location_method = widgets.VBox([city_name])
analysis_date_picker = widgets.DatePicker(description='Data da Análise:', value=date.today())
min_altitude_slider = widgets.IntSlider(value=30, min=10, max=90, step=5, description='Elev. Mínima (°):')
targets_method = widgets.RadioButtons(options=['Lista Fixa', 'Sistema Solar', 'Manual'], description='Alvos:', value='Lista Fixa', layout={'width': 'max-content'})
manual_targets_input = widgets.Textarea(description='Nomes:', placeholder='Sirius\nBetelgeuse\n... (um por linha)')
box_targets_method = widgets.VBox([widgets.Label(value="Nomes dos alvos de céu profundo pré-selecionados.")])
run_button = widgets.Button(description='Gerar Análise Noturna', button_style='success', icon='check')
output_area_nightly = widgets.Output()

# --- Layout da UI (como antes) ---
location_box = widgets.VBox([location_method, box_location_method])
config_box = widgets.VBox([analysis_date_picker, min_altitude_slider])
targets_box = widgets.VBox([targets_method, box_targets_method])
ui_nightly = widgets.VBox([
    widgets.HTML("<h3>Configurações da Análise:</h3>"),
    widgets.HBox([location_box, config_box, targets_box]),
    run_button
])

# --- Callbacks e Funções (como antes, com pequenas melhorias) ---
def on_location_method_change(change):
    if change.new == 'Cidade':
        box_location_method.children = [city_name]
    else:
        box_location_method.children = [lat_lon_input]
location_method.observe(on_location_method_change, names='value')

def on_targets_method_change(change):
    if change.new == 'Manual':
        box_targets_method.children = [manual_targets_input]
    elif change.new == 'Lista Fixa':
        box_targets_method.children = [widgets.Label(value="Nomes dos alvos de céu profundo pré-selecionados.")]
    else: # Sistema Solar
        box_targets_method.children = [widgets.Label(value="Planetas, Sol, Lua e Plutão.")]
targets_method.observe(on_targets_method_change, names='value')

def create_interactive_sky_map(targets_coords, observer_location, night_events, observer_timezone):
    # ... (código do mapa como antes) ...

def on_run_button_clicked(b):
    with output_area_nightly:
        # ... (lógica da análise noturna como antes) ...

run_button.on_click(on_run_button_clicked)

---## 2. Análise Anual (Calendário de Visibilidade)

In [ ]:
# --- Widgets da Análise Anual ---
yearly_target_name = widgets.Text(description='Alvo:', placeholder='Ex: M31')
yearly_year_picker = widgets.IntText(value=date.today().year, description='Ano:')
run_button_yearly = widgets.Button(description='Gerar Calendário Anual', button_style='info', icon='calendar')
output_area_yearly = widgets.Output()

# --- Layout da UI Anual ---
ui_yearly = widgets.VBox([
    widgets.HTML("<h3>Configurações do Calendário:</h3>"),
    widgets.HBox([yearly_target_name, yearly_year_picker]),
    run_button_yearly
])

# --- Callback do Botão Anual ---
def on_run_button_yearly_clicked(b):
    with output_area_yearly:
        clear_output(wait=True)
        print(f"Iniciando análise anual para '{yearly_target_name.value}' em {yearly_year_picker.value}...")
        
        # 1. Obter Localização (reaproveitando da UI de cima)
        try:
            if location_method.value == 'Cidade':
                if not city_name.value: raise ValueError("Nome da cidade não preenchido na seção 1.")
                observer_location = get_location_from_city(city_name.value)
            else:
                if not lat_lon_input.value: raise ValueError("Coordenadas não preenchidas na seção 1.")
                lat, lon = map(float, lat_lon_input.value.split(','))
                observer_location = EarthLocation(lat=lat*u.deg, lon=lon*u.deg)
            observer_timezone = set_timezone_for_sao_paulo(observer_location)
            print(f"Localização definida: {observer_location}")
        except Exception as e:
            print(f"Erro ao obter localização: {e}")
            return
            
        # 2. Obter Alvo
        try:
            target_name = yearly_target_name.value
            if not target_name: raise ValueError("Nome do alvo não preenchido.")
            target_coord = SkyCoord.from_name(target_name)
            print(f"Alvo definido: {target_name}")
        except Exception as e:
            print(f"Erro ao obter coordenadas do alvo: {e}")
            return
            
        # 3. Executar Análise Anual
        year = yearly_year_picker.value
        min_altitude = min_altitude_slider.value * u.deg
        
        df_year = analyze_year_visibility(year, observer_location, observer_timezone, target_coord, min_altitude)
        
        # 4. Plotar o Calendário
        if not df_year.empty:
            print("Plotando o calendário de visibilidade...")
            plot_yearly_visibility(df_year, target_name, year)
        else:
            print(f"Nenhum período de visibilidade encontrado para {target_name} em {year} com os critérios definidos.")

run_button_yearly.on_click(on_run_button_yearly_clicked)


## 3. Execução

In [ ]:
display(ui_nightly, output_area_nightly)
display(widgets.HTML('<hr>')) # Separador visual
display(ui_yearly, output_area_yearly)